In [ ]:
# run the script initnotebook.py in the cuurent folder
# Error when run multiple times becasue the directory changed
%run initnotebook.py

In [ ]:
import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from config_cityscapes import *
import os
from dataloader.cityscapes_dataloader import CityscapesDataset
from val_segformer_rgbonly import val_cityscape

In [ ]:
config_path = 'dataloader/cityscapes_rgbd_config.yaml'
config_path = os.path.join(projectFolder, config_path)

cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

data_mean = [0.291,  0.329,  0.291]
data_std = [0.190,  0.190,  0.185]

In [ ]:
cityscapes_test = CityscapesDataset(cfg, split='val')
test_loader = DataLoader(cityscapes_test, batch_size=1, shuffle=False, num_workers=4) # batchsize?
print(f'total test sample: {len(cityscapes_test)} v_iteration:{len(test_loader)}')


In [ ]:
pretrained_model_path = './pretrained/model_400.pth'
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)

model = segmodel(cfg=config, criterion=criterion, norm_layer=nn.BatchNorm2d, test=True)
model = nn.DataParallel(model, device_ids = config.device_ids)
model.to(f'cuda:{model.device_ids[0]}', non_blocking=True)

In [ ]:
saved_model_path = os.path.join(projectFolder, pretrained_model_path)
print(saved_model_path)
# exit()
state_dict = torch.load(saved_model_path)
model.load_state_dict(state_dict['model'], strict=True)
print(f'model loaded')
epoch = state_dict['epoch']


In [ ]:
val_loss, val_mean_iou = val_cityscape(epoch, test_loader, model)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Access the first batch from the DataLoader
for batch in val_loader:
    images = batch['image']

    # Process each image in the batch
    for i in range(images.shape[0]):
        img = images[i].numpy().transpose((1, 2, 0))
        img *= data_std
        img += data_mean

        print(img.shape)
        # Plot the image in the subplot
        plt.subplot(1, 4, i + 1)  # 1 row, 4 columns, i+1 is the current subplot
        plt.imshow(img)
        plt.axis('off')

    plt.show()

    # Break the loop after the first batch
    break


In [ ]:
print(cfg)